# 掩码语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_func(examples):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer,   #要对源文本掩码，所以该位置得到的ids是103
    # 启用掩码语言模型
    mlm=True,
    # 掩码概率15%
    mlm_probability=0.15))

In [8]:
next(enumerate(dl))
#enumerate用于将一个可遍历的数据对象(如列表、元组或字符串)及其索引（从零开始）组合成一个个元组，返回一个迭代器，每个元组包含两个元素：当前元素的索引和元素本身
#next()用于从迭代器enumerate中获取下一个项目

(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,  6858,   103,  2110,  1300,  4289, 13465,
           8020,   103,   112,  9064, 12095,   103,  8626,   103,  8736,   103,
           8021,  3221,   671,  2429,   855,   103,  6205,  2128,   769,  6858,
           1920,   103,  4638,  1300,  4289,   103,  8024,  7667,  7270,   103,
           7247,  3209,  1587,   511,  1325,  1259,  8258,  2399,   130,  3299,
            103,   103,   103,  1993,  5040,  2456,  8024,  8138,  2399,   103,
           3299,   129,  3189,  3633,   103,  2456,  2768,  2458,  7667,  8024,
            855,   754,  6205,  2128,   769,  6858,  1920,   103,  1069,  2412,
           3413,   103,  7362,  6205,  4689,   103,  2128,  2356,   103,  2123,
           6205,  6662,   103,  1384,   511,  2456,  5029,  7481,  4916,   103,
            117,  8280,  2398,  5101,  8024,  2245,  3133,  7481,  4916,   125,
            117,  8195,  2398,  5101,   103,  7667,  5966,  3152,  4289,   125,
            117,  8567

In [9]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

## Step4 创建模型

In [10]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Step5 配置训练参数

In [11]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [12]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

## Step7 模型训练

In [19]:
trainer.train()

Step,Training Loss
10,0.678500
20,0.855400
30,0.936200
40,1.172600
50,1.373000
60,1.341100
70,1.432300
80,1.334000
90,1.339300
100,1.411000


TrainOutput(global_step=313, training_loss=1.2757719979880335, metrics={'train_runtime': 4079.4462, 'train_samples_per_second': 2.451, 'train_steps_per_second': 0.077, 'total_flos': 1973819658240000.0, 'train_loss': 1.2757719979880335, 'epoch': 1.0})

## Step8 模型推理

In [20]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [21]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9979416728019714,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.000883428961969912,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.00011019018711522222,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.000105615436041262,
   'token': 769,
   'token_str': '交',
   'sequence': "[CLS] 西 安 交 通 交 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 6.960154132684693e-05,
   'token': 2339,
   'token_str': '工',
   'sequence': "[CLS] 西 安 交 通 工 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.996500968933105

In [22]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.19138303399085999,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.07672032713890076,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04008426517248154,
   'token': 6566,
   'token_str': '负',
   'sequence': '[CLS] 下 面 是 一 则 负 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03435131162405014,
   'token': 2207,
   'token_str': '小',
   'sequence': '[CLS] 下 面 是 一 则 小 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.024878691881895065,
   'token': 3952,
   'token_str': '游',
   'sequence': '[CLS] 下 面 是 一 则 游 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.08860170841217041,
   'token': 4638,
   'token_str': '的',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 的 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  